# Functions

In [1]:
def search_term(text,term):
    text=clean_text(text) #remove some chars and apply lowercase
    if ' ' in term:
        return term.lower() in text
    else:
        import re
        term='%s' %(term.lower().strip())
        pm='[\s(]%s[,.$\s\?!:)]|^%s[,.$\s\?!:]'%(term,term) #pattern 
        occ_term=len(re.findall(pm,text,re.MULTILINE))
        return occ_term>0
    
def clean_text(text):
    for el in ['\n',';','.',',',':','!','(',')','?']:
        text=text.replace(el,' ')
    text=" %s " %(text)
    return text.lower()

def process_count_re(df):
    def clean_text(text):
        for el in ['\n',';','.',',',':','!','(',')','?']:
            text=text.replace(el,' ')
        text=" %s " %(text)
        return text.lower()
    import pandas as pd
    import re
    def search_term(text,term):
        if ' ' in term:
            return term in text
        else:
            import re
            term='%s' %(term.lower().strip())
            pm='[\s(]%s[,.$\s\?!:)]|^%s[,.$\s\?!:]'%(term,term) #pattern 
            occ_term=len(re.findall(pm,text,re.MULTILINE))
            return occ_term>0
        
    #import keywords
    import pandas as pd
    df_terms=pd.read_excel('./keywords_final.xlsx')
    df_terms={key:list(df_terms[key].dropna().values) for key in list(df_terms.keys())}


    results=[]
    for el in df[['id', 'sentences']].values:
        id_=el[0]
        text=clean_text(el[1])
        
        result=[]
        result.append(id_)
        result.append(el[1])
        present=False
        for topic in list(df_terms.keys()):
            t=False
            term_find=''
            for term in df_terms[topic]:
                res=search_term(text,term)
                if res:
                    t=True
                    term_find=term
                    present=True
                    break #we stop the search if one term is found, for computational reduction
            result.append(t) 
            result.append(term_find) 
        
        if present: #if at least one topic is present we save the result
            results.append(result)
    return results

def process_keywords(df_terms,col,text):
    terms=df_terms[col]
    results=''
    for term in terms:
        if search_term(text,term):
            if results=='':
                results=term
            else:
                results='%s|%s'%(results,term)
    return results


#function to evaluate rules
def evaluate_rules(input_string):
    import re
    pattern = re.compile(r'\((.*?)\)')
    matches = pattern.findall(input_string)
    result = []

    for match in matches:
        components = [c.strip() for c in match.split('&')]
        result.append('input_data[\'{}\']'.format('\'] & input_data[\''.join(components)))

    remaining_string = pattern.sub('', input_string)
    remaining_string = remaining_string.replace('|', '').strip()
    if remaining_string:
        result.append('input_data[\'{}\']'.format(remaining_string))

    return ' | '.join(result)

# Run seta to generate preliminary list of terms

In [ ]:
#https://seta.jrc.ec.europa.eu/profile/key-generation-instructions how to generate priv key
#link to register application https://seta.jrc.ec.europa.eu/login?redirect=/profile/applications

In [2]:
def get_token():
    import sys
    import random
    import string

    import requests


    AUTH_URL = "https://seta.jrc.ec.europa.eu/authentication/v1/token"

    # request token with authorization
    payload = {
        "username": "",
        "provider": "",
        "rsa_original_message": '',
        "rsa_message_signature": '',
    }
    token_request = requests.post(AUTH_URL, json=payload, timeout=30)

    if not token_request.ok:
        print("auth failed: %s", token_request.text)

    import json
    token=json.loads(token_request.text)
    return token
token=get_token()
token


import requests
session = requests.Session()
term='open source software'
payload = dict(term=term)
headers = {"Authorization": ("Bearer " + token['access_token'])}
API_URL = "https://seta.jrc.ec.europa.eu/seta-search/api/v1/"
api_url=API_URL
print(api_url)
#print(token['access_token'])
r = session.get(api_url + "ontology-list", params=payload, headers=headers)

r.json()

https://seta.jrc.ec.europa.eu/seta-search/api/v1/


{'nodes': [['open source'],
  ['free software', 'open content'],
  ['open standards',
   'ict standards',
   'industry standards',
   'interoperable solutions',
   'ict solutions'],
  ['web services',
   'middleware',
   'web applications',
   'web service',
   'apis',
   'software applications',
   'ontologies',
   'software components',
   'grid middleware',
   'architectures',
   'application software',
   'operating systems',
   'mobile devices',
   'web-based',
   'web application'],
  ['open source tools', 'software tools'],
  ['cloud services',
   'cloud computing',
   'digital services',
   'cloud solutions',
   'cloud computing services',
   'cloud service',
   'online services',
   'internet-based services',
   'big data',
   'egovernment',
   'social computing',
   'e-commerce',
   'big data analytics',
   'e-business'],
  ['proprietary software',
   'commercial software',
   'proprietary',
   'proprietary solutions'],
  ['open data', 'public data', 'open government', 'publi

# Importing Data

In [2]:
import pandas as pd
output_file='./b2g_es_award_notice_max5cpv.csv'
df=pd.read_csv(output_file)
df['year']=df['id'].apply(lambda x:x.split('/')[0])
df.head()

,id,_score,sentences,sentences_nr,url,main_cpv_code,main_cpv_description,euro,iso_country,document_type,...,second_cpv_code,second_cpv_code_description,third_cpv_code,third_cpv_code_description,fourth_cpv_code,fourth_cpv_code_description,fifth_cpv_code,fifth_cpv_code_description,nr_cpv,year
0,2021/S 194-505039,0.904236,Märkische Kliniken GmbH|Paulmannshöher Straße ...,14,https://ted.europa.eu/udl?uri=TED:NOTICE:50503...,33112000,"Echo, ultrasound and doppler imaging equipment",2237044.0,DE,Contract award notice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2021
1,2022/S 010-020697,0.904236,NEW ZAGORA STATE FORESTRY DEPARTMENT|Str. PETK...,14,https://ted.europa.eu/udl?uri=TED:NOTICE:02069...,50110000,Repair and maintenance services of motor vehic...,11350.0,BG,Contract award notice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2022
2,2021/S 131-345804,0.904236,Hatanpää Highway 30|Magnetic foot and command ...,11,https://ted.europa.eu/udl?uri=TED:NOTICE:34580...,32352000,Aerials and reflectors,310346.0,FI,Contract award notice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2021
3,2021/S 038-095798,0.904236,National Revenue Agency|52 Knyaz Aleksandar Do...,19,http://ted.europa.eu/udl?uri=TED:NOTICE:095798...,66172000,Financial transaction processing and clearing-...,3162223.0,BG,Contract award notice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2021
4,2021/S 132-350480,0.904236,Hessian Data Processing Centre|Mainzer Road 29...,31,https://ted.europa.eu/udl?uri=TED:NOTICE:35048...,48600000,Database and operating software package,436426.0,DE,Contract award notice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2021


In [3]:
df.groupby(['year'])['id'].count()

year
2021    249555
2022    275394
Name: id, dtype: int64

# Searching keywords

## Parallel process

In [5]:
import numpy as np
import pandas as pd
import multiprocessing as mp
import datetime
from pathos.multiprocessing import ProcessingPool as Pool
cores=mp.cpu_count()

#import keywords
import pandas as pd
df_terms=pd.read_excel('./keywords_final.xlsx')
df_terms={key:list(df_terms[key].dropna().values) for key in list(df_terms.keys())}
#setting colnames
cols=['id','text']
for col in df_terms.keys():
    cols.append(col)
    cols.append('%s-terms'%(col))

matching=[]

start_row=0
buffer_size=50000
print (datetime.datetime.now(),'processing',len(df),'rows')
#we have issue with memory allocation, for this reason I adopt this slice looping
for end_row in list(np.arange(buffer_size,(int(len(df)/buffer_size)+2)*buffer_size,buffer_size)):

    df_=df.iloc[start_row:end_row].reset_index()
    del(df_['index'])
    df_=df_[['id','sentences']]
    print (datetime.datetime.now(),'running from',start_row,'to',end_row, 'on #rows:',len(df_))
    df_split = np.array_split(df_, cores, axis=0)
    
    # create the multiprocessing pool
    pool = Pool(cores)
    # process the DataFrame by mapping function to each df across the pool
    df_out = np.vstack(pool.map(process_count_re, df_split))
    # close down the pool and join
    pool.close()
    pool.join()
    pool.clear()
    df_split=None
    df_=None
    
    results=pd.DataFrame(df_out)
    df_out=None
    results.columns=cols
    matching.append(results)
   
    #update start_row
    start_row=end_row
matching=pd.concat(matching)
print (datetime.datetime.now(),'processed',len(df),'rows')

#exporting results
output_file='./paper.xlsx'
matching.to_excel(output_file,index=False)
matching

2024-06-17 15:53:50.775406 processing 524949 rows
2024-06-17 15:53:50.840399 running from 0 to 50000 on #rows: 50000
2024-06-17 15:54:39.668365 running from 50000 to 100000 on #rows: 50000
2024-06-17 15:56:02.541136 running from 100000 to 150000 on #rows: 50000
2024-06-17 15:57:13.324183 running from 150000 to 200000 on #rows: 50000
2024-06-17 15:58:08.689284 running from 200000 to 250000 on #rows: 50000
2024-06-17 15:58:58.563736 running from 250000 to 300000 on #rows: 50000
2024-06-17 15:59:48.370014 running from 300000 to 350000 on #rows: 50000
2024-06-17 16:00:36.303053 running from 350000 to 400000 on #rows: 50000
2024-06-17 16:01:23.532986 running from 400000 to 450000 on #rows: 50000
2024-06-17 16:02:11.156594 running from 450000 to 500000 on #rows: 50000
2024-06-17 16:03:02.303315 running from 500000 to 550000 on #rows: 24949
2024-06-17 16:03:28.282952 processed 524949 rows


,id,text,OPEN SOURCE SOFTWARE,OPEN SOURCE SOFTWARE-terms,OPEN STANDARDS,OPEN STANDARDS-terms,GEOSPATIAL STANDARDS,GEOSPATIAL STANDARDS-terms,OPEN DATA,OPEN DATA-terms,OPEN GEOSPATIAL DATASETS,OPEN GEOSPATIAL DATASETS-terms,GEOSPATIAL,GEOSPATIAL-terms,GIS OPEN SOURCE SOFTWARE,GIS OPEN SOURCE SOFTWARE-terms,PROPRIETARY GEOSPATIAL SOFTWARE,PROPRIETARY GEOSPATIAL SOFTWARE-terms
0,2022/S 210-604006,Purchase of support services for spatial infor...,False,,False,,False,,False,,False,,True,geographic information,False,,False,
1,2021/S 250-663773,Dachauer Str. 128|§ 99(2) GWB contracting auth...,True,linux,False,,False,,False,,False,,False,,False,,False,
2,2022/S 246-712799,Council of Torrevieja City Council|Constitutiv...,False,,False,,False,,False,,False,,False,,True,GEOS,False,
3,2022/S 152-435218,Centre of Telecomunicacions i Tecnologies de l...,False,,False,,False,,True,open government,False,,False,,False,,False,
4,2021/S 211-557243,SEVILLE METROPOLITAN WATER SUPPLY AND SANITATI...,False,,False,,False,,False,,False,,True,geographical information,False,,False,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,2021/S 144-382523,University Hospital in Krakow|U.S.A. Kopernika...,False,,False,,False,,False,,False,,True,copernicus,False,,False,
312,2021/S 186-484254,Federal Ministry for Economic Cooperation and ...,False,,False,,False,,True,open data,False,,False,,False,,False,
313,2022/S 040-104009,European Centre for Medium-Range Weather Forec...,False,,True,interoperable,False,,False,,False,,True,copernicus,False,,False,
314,2022/S 033-086175,Directorate for Home Affairs and Justice of th...,False,,False,,False,,False,,False,,True,geoinformation,False,,False,


In [6]:
matching[matching['text'].apply(lambda x: 'open dataset' in x)].text.values

array(['Rue du Nord 76 / Noordstraat 76|Open Maps for Europe: The Development and Enhancement of a User Interface|Deliver an enhanced gateway with access to discover, view, license and download open datasets. The interface will include a map viewer and search functionality. Establish a new production process and provide a stable service of open cadastral data.|The Open Maps for Europe Interface|• Deliver an enhanced gateway with access to discover, view, license and download open datasets. The interface will include a map viewer and search functionality.|• Ensure that a first iteration of the user interface will be available 6 months after the award of the contract. Incremental developments to take place during the course of the project until October 2022 when a final version must be available.|• Establish a production process and provide the following:|— Web Feature Services (OGC WFS) for accessing open datasets,|— Web Map Services (OGC WMS) for viewing styled maps (e.g., in geoportal

In [7]:
matching[matching['text'].apply(lambda x: 'open dataset' in x)]

,id,text,OPEN SOURCE SOFTWARE,OPEN SOURCE SOFTWARE-terms,OPEN STANDARDS,OPEN STANDARDS-terms,GEOSPATIAL STANDARDS,GEOSPATIAL STANDARDS-terms,OPEN DATA,OPEN DATA-terms,OPEN GEOSPATIAL DATASETS,OPEN GEOSPATIAL DATASETS-terms,GEOSPATIAL,GEOSPATIAL-terms,GIS OPEN SOURCE SOFTWARE,GIS OPEN SOURCE SOFTWARE-terms,PROPRIETARY GEOSPATIAL SOFTWARE,PROPRIETARY GEOSPATIAL SOFTWARE-terms
121,2021/S 011-022934,Rue du Nord 76 / Noordstraat 76|Open Maps for ...,False,,False,,True,OGC,True,open data,False,,True,web map,False,,False,


## Complete keywords

In [8]:
import datetime
import pandas as pd
output_file='./paper.xlsx'
input_data=pd.read_excel(output_file)
print(datetime.datetime.now(),'importing results:',len(input_data))

#import keywords
import pandas as pd
df_terms=pd.read_excel('./keywords_final.xlsx')
df_terms={key:list(df_terms[key].dropna().values) for key in list(df_terms.keys())}

#complete list of keywords
for col in list(df_terms.keys()):
    print(datetime.datetime.now(),'enriching',col)
    input_data[col+'-terms-full']=input_data['text'].apply(lambda x: process_keywords(df_terms,col,x))

input_data.to_excel(output_file,index=False)
input_data.head()

2024-06-17 16:03:34.646681 importing results: 6323
2024-06-17 16:03:34.688121 enriching OPEN SOURCE SOFTWARE
2024-06-17 16:03:44.882200 enriching OPEN STANDARDS
2024-06-17 16:03:50.927681 enriching GEOSPATIAL STANDARDS
2024-06-17 16:04:50.881976 enriching OPEN DATA
2024-06-17 16:04:55.727658 enriching OPEN GEOSPATIAL DATASETS
2024-06-17 16:04:59.256392 enriching GEOSPATIAL
2024-06-17 16:05:18.485716 enriching GIS OPEN SOURCE SOFTWARE
2024-06-17 16:06:13.385831 enriching PROPRIETARY GEOSPATIAL SOFTWARE


,id,text,OPEN SOURCE SOFTWARE,OPEN SOURCE SOFTWARE-terms,OPEN STANDARDS,OPEN STANDARDS-terms,GEOSPATIAL STANDARDS,GEOSPATIAL STANDARDS-terms,OPEN DATA,OPEN DATA-terms,...,PROPRIETARY GEOSPATIAL SOFTWARE,PROPRIETARY GEOSPATIAL SOFTWARE-terms,OPEN SOURCE SOFTWARE-terms-full,OPEN STANDARDS-terms-full,GEOSPATIAL STANDARDS-terms-full,OPEN DATA-terms-full,OPEN GEOSPATIAL DATASETS-terms-full,GEOSPATIAL-terms-full,GIS OPEN SOURCE SOFTWARE-terms-full,PROPRIETARY GEOSPATIAL SOFTWARE-terms-full
0,2022/S 210-604006,Purchase of support services for spatial infor...,False,NaN,False,NaN,False,NaN,False,NaN,...,False,NaN,,,,,,geographic information|geographic information ...,,
1,2021/S 250-663773,Dachauer Str. 128|§ 99(2) GWB contracting auth...,True,linux,False,NaN,False,NaN,False,NaN,...,False,NaN,linux,,,,,,,
2,2022/S 246-712799,Council of Torrevieja City Council|Constitutiv...,False,NaN,False,NaN,False,NaN,False,NaN,...,False,NaN,,,,,,,GEOS,
3,2022/S 152-435218,Centre of Telecomunicacions i Tecnologies de l...,False,NaN,False,NaN,False,NaN,True,open government,...,False,NaN,,,,open government,,,,
4,2021/S 211-557243,SEVILLE METROPOLITAN WATER SUPPLY AND SANITATI...,False,NaN,False,NaN,False,NaN,False,NaN,...,False,NaN,,,,,,geographical information|gis,,


## Apply rules

In [9]:
import pandas as pd
output_file='./paper.xlsx'
input_data=pd.read_excel(output_file)
print(datetime.datetime.now(),'importing enriched results:',len(input_data))

#import rules
import pandas as pd
df_rules=pd.read_excel('./keywords_final.xlsx',sheet_name='rules')
print(df_rules)

topics_cols=['id','text']
for row in df_rules.values:
    topic=row[0]
    rule=row[1]
    rule = evaluate_rules(rule)
    input_data[topic]=eval(rule)
    topics_cols.append(topic)
input_data.to_excel(output_file,index=False)
input_data[topics_cols]

2024-06-17 16:06:45.206256 importing enriched results: 6323
                             TOPIC  \
0  Open source geospatial software   
1        Open geospatial standards   
2             Open geospatial data   
3             Open source software   
4                   Open standards   
5                        Open data   
6  Proprietary geospatial software   

                                KEYWORDS OF INTEREST  \
0  (OPEN SOURCE SOFTWARE & GEOSPATIAL) | GIS OPEN...   
1  (OPEN STANDARDS & GEOSPATIAL) | GEOSPATIAL STA...   
2  (OPEN DATA & GEOSPATIAL) | (OPEN GEOSPATIAL DA...   
3  (OPEN SOURCE SOFTWARE) | (GIS OPEN SOURCE SOFT...   
4          (OPEN STANDARDS) | (GEOSPATIAL STANDARDS)   
5           (OPEN DATA) | (OPEN GEOSPATIAL DATASETS)   
6                  (PROPRIETARY GEOSPATIAL SOFTWARE)   

                                        GPT question  
0  Does the content of the following text focus o...  
1  Does the content of the following text focus o...  
2  Does the content o

,id,text,Open source geospatial software,Open geospatial standards,Open geospatial data,Open source software,Open standards,Open data,Proprietary geospatial software
0,2022/S 210-604006,Purchase of support services for spatial infor...,False,False,False,False,False,False,False
1,2021/S 250-663773,Dachauer Str. 128|§ 99(2) GWB contracting auth...,False,False,False,True,False,False,False
2,2022/S 246-712799,Council of Torrevieja City Council|Constitutiv...,True,False,False,True,False,False,False
3,2022/S 152-435218,Centre of Telecomunicacions i Tecnologies de l...,False,False,False,False,False,True,False
4,2021/S 211-557243,SEVILLE METROPOLITAN WATER SUPPLY AND SANITATI...,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
6318,2021/S 144-382523,University Hospital in Krakow|U.S.A. Kopernika...,False,False,False,False,False,False,False
6319,2021/S 186-484254,Federal Ministry for Economic Cooperation and ...,False,False,False,False,False,True,False
6320,2022/S 040-104009,European Centre for Medium-Range Weather Forec...,False,True,False,False,True,False,False
6321,2022/S 033-086175,Directorate for Home Affairs and Justice of th...,False,False,False,False,False,False,False


In [10]:
res=[]
for topic_col in topics_cols:
    r=pd.DataFrame(input_data[input_data[topic_col]==True].groupby([topic_col])['id'].count())
    r.columns=['count']
    r['topic']=topic_col
    res.append(r)
res=pd.concat(res)
res[['topic','count']]

,topic,count
True,Open source geospatial software,110
True,Open geospatial standards,197
True,Open geospatial data,24
True,Open source software,1347
True,Open standards,1372
True,Open data,461
True,Proprietary geospatial software,299


# GPT classification

In [8]:
import pandas as pd
output_file='./paper.xlsx'
input_data=pd.read_excel(output_file)
input_data.head()

,id,text,OPEN SOURCE SOFTWARE,OPEN SOURCE SOFTWARE-terms,OPEN STANDARDS,OPEN STANDARDS-terms,GEOSPATIAL STANDARDS,GEOSPATIAL STANDARDS-terms,OPEN DATA,OPEN DATA-terms,...,GEOSPATIAL-terms-full,GIS OPEN SOURCE SOFTWARE-terms-full,PROPRIETARY GEOSPATIAL SOFTWARE-terms-full,Open source geospatial software,Open geospatial standards,Open geospatial data,Open source software,Open standards,Open data,Proprietary geospatial software
0,2022/S 210-604006,Purchase of support services for spatial infor...,False,NaN,False,NaN,False,NaN,False,NaN,...,geographic information|geographic information ...,NaN,NaN,False,False,False,False,False,False,False
1,2021/S 250-663773,Dachauer Str. 128|§ 99(2) GWB contracting auth...,True,linux,False,NaN,False,NaN,False,NaN,...,NaN,NaN,NaN,False,False,False,True,False,False,False
2,2022/S 246-712799,Council of Torrevieja City Council|Constitutiv...,False,NaN,False,NaN,False,NaN,False,NaN,...,NaN,GEOS,NaN,True,False,False,True,False,False,False
3,2022/S 152-435218,Centre of Telecomunicacions i Tecnologies de l...,False,NaN,False,NaN,False,NaN,True,open government,...,NaN,NaN,NaN,False,False,False,False,False,True,False
4,2021/S 211-557243,SEVILLE METROPOLITAN WATER SUPPLY AND SANITATI...,False,NaN,False,NaN,False,NaN,False,NaN,...,geographical information|gis,NaN,NaN,False,False,False,False,False,False,False


In [9]:
#import questions
import pandas as pd
df_questions=pd.read_excel('./keywords_final.xlsx',sheet_name='rules')
df_contexts={row[0]:row[3] for row in df_questions.values}
df_questions={row[0]:row[2] for row in df_questions.values}
df_questions

{'Open source geospatial software': 'Does the content of the following text focus on using, or providing services based on GIS or geospatial open source software?',
 'Open geospatial standards': 'Does the content of the following text focus on the use or development of open geospatial standards?',
 'Open geospatial data': 'Does the content of the following text focus on the sharing or use of open geospatial data?',
 'Open source software': 'Does the content of the following text focus on using, or providing services based on open source software?',
 'Open standards': 'Does the content of the following text focus on the use or development of open standards?',
 'Open data': 'Does the content of the following text focus on the sharing or use of open data?',
 'Proprietary geospatial software': 'Does the content of the following text focus on using, or providing services based on GIS or geospatial proprietary software?'}

In [ ]:
import openai
import datetime
import pandas as pd
import time
TOKEN = ""
openai.api_base = "https://api-gpt.jrc.ec.europa.eu/v1"
openai.api_key = TOKEN
results={}

errors=0
models=['nous-hermes-2-mixtral-8x7b-dpo']
input_data=pd.read_excel(output_file)
for topic in df_questions:
    question=df_questions[topic]
    context=df_contexts[topic]
    for model in models[:]:
        print(datetime.datetime.now(),'run',model)
        gpt_results=[]
        rows=input_data.values
        rows=input_data[input_data[topic]==True][['id','text',topic]].values
        print(datetime.datetime.now(),'processing %s rows for the question "%s"' %(len(rows),question))
        for row in rows:
            time.sleep(3)
            id=row[0]
            sentence=row[1]
            label=0 if row[2]==False else 1
            try:
                chat_completion = openai.ChatCompletion.create(
                headers = {"Authorization": f"Bearer {TOKEN}"},
                model = model,
                temperature=0.1,
                stream = False,
                messages = [{"role":"system","content":context},
                    {"role": "user", "content": "%s, answer only yes or no and give a probability of a yes, I want a reply with 3 elements separed by semicolon like this <(YES/NO);probability;comment> :%s" %(question,sentence)}]
                )
            except Exception as e:
                print("An exception occurred:", str(e))
                print('-------')
                print('error for',id)
                errors+=1
                print('-------')
                continue

            # print the chat completion
            #print(chat_completion.choices[0].message.content,'-> id:',id,',original label:',label)

            predicted_label=None
            content=chat_completion.choices[0].message.content
            content=content.split(';')
            chatGPT_answer=content[0]
            chatGPT_probability=None
            comments=''
            if len(content)>1:
                chatGPT_probability=content[1]
            if len(content)>2:
                comments=content[2]
            if 'YES' in chatGPT_answer.upper():
                predicted_label=1
            elif 'NO' in chatGPT_answer.upper():
                predicted_label=0
            gpt_results.append((id,sentence,label,predicted_label,chatGPT_answer,chatGPT_probability,comments,question)) #change 1 to label
            #print('/////////////////////////')

        gpt_results=pd.DataFrame(gpt_results)
        gpt_results.columns=['id','sentences','labels','chatGPT_labels','chatGPT_answer','chatGPT_probability','chatGPT_comments','question']
        gpt_results=gpt_results.dropna()
        results[(model,question)]={'annotated_dataset':gpt_results}

2024-06-19 08:10:41.248711 run nous-hermes-2-mixtral-8x7b-dpo
2024-06-19 08:10:41.269891 processing 110 rows for the question "Does the content of the following text focus on using, or providing services based on GIS or geospatial open source software?"
2024-06-19 08:17:46.836370 run nous-hermes-2-mixtral-8x7b-dpo
2024-06-19 08:17:46.841596 processing 197 rows for the question "Does the content of the following text focus on the use or development of open geospatial standards?"
2024-06-19 08:30:15.300184 run nous-hermes-2-mixtral-8x7b-dpo
2024-06-19 08:30:15.304742 processing 24 rows for the question "Does the content of the following text focus on the sharing or use of open geospatial data?"
2024-06-19 08:31:48.062018 run nous-hermes-2-mixtral-8x7b-dpo
2024-06-19 08:31:48.067901 processing 1347 rows for the question "Does the content of the following text focus on using, or providing services based on open source software?"
2024-06-19 09:58:01.767838 run nous-hermes-2-mixtral-8x7b-dpo

In [15]:
all_gpt_results=[]
for gpt in results:
    print(len(results[gpt]['annotated_dataset']))
    all_gpt_results.append(results[gpt]['annotated_dataset'])
all_gpt_results=pd.concat(all_gpt_results)
all_gpt_results

110
197
24
1347
1371
461
299


,id,sentences,labels,chatGPT_labels,chatGPT_answer,chatGPT_probability,chatGPT_comments,question
0,2022/S 246-712799,Council of Torrevieja City Council|Constitutiv...,1,0,(NO),0%,The text provided does not focus on using or p...,Does the content of the following text focus o...
1,2021/S 211-552688,Dózsa György út 41.|OMRRK Materials and Techno...,1,0,(NO),0%,The text provided does not focus on using or p...,Does the content of the following text focus o...
2,2022/S 105-293916,"City of Gliwice, Department of Public Procurem...",1,0,(NO),0%,The text provided does not focus on using or p...,Does the content of the following text focus o...
3,2021/S 189-491481,Research centre Jülich GmbH|Purchasing and mat...,1,0,(NO),0%,The text does not focus on using or providing ...,Does the content of the following text focus o...
4,2022/S 006-012445,Municipality of Dąbrowa Górnicza|U.S.A. Bounda...,1,0,(NO),0%,The text provided does not focus on using or p...,Does the content of the following text focus o...
...,...,...,...,...,...,...,...,...
294,2021/S 081-209781,Chair of the Board of Directors of the Port Au...,1,1,(YES,80%,) The text focuses on using and providing serv...,Does the content of the following text focus o...
295,2021/S 124-329242,Freedom Square 7|ESRI ArcGIS software purchase...,1,1,(YES,80%,The text mentions the purchase and maintenance...,Does the content of the following text focus o...
296,2022/S 143-410171,Bavarian State Forests AöR|Forest planning 4.0...,1,1,(YES,90%,) The text focuses on using and providing serv...,Does the content of the following text focus o...
297,2021/S 041-103651,Inter-municipal Society of Flanders for Water ...,1,1,(YES,90%,) The content of the text focuses on using and...,Does the content of the following text focus o...


In [16]:
all_gpt_results.to_excel('./paper_gpt_validation_w_context.xlsx',index=False)

In [17]:
all_gpt_results[all_gpt_results['chatGPT_labels']==1]

,id,sentences,labels,chatGPT_labels,chatGPT_answer,chatGPT_probability,chatGPT_comments,question
11,2022/S 248-718143,Austrian Institute for Transport Data Infrastr...,1,1,(YES),0.8,The text mentions the development of GIP 2.0 c...,Does the content of the following text focus o...
20,2022/S 184-521320,Federal Institute for Geosciences and Raw Mate...,1,1,(YES),80%,The text focuses on the development and applic...,Does the content of the following text focus o...
27,2021/S 008-014516,Mayor of Oleiros City Council|Square of Galici...,1,1,(YES),80%,The text mentions the maintenance of the corpo...,Does the content of the following text focus o...
32,2021/S 232-611606,Federal Institute for Geosciences and Raw Mate...,1,1,(YES),90%,The text focuses on the development of a platf...,Does the content of the following text focus o...
37,2021/S 189-492336,President of the Provincial Council of Valenci...,1,1,(YES),90%,The text focuses on providing services based o...,Does the content of the following text focus o...
...,...,...,...,...,...,...,...,...
292,2022/S 170-482481,Science Park 8|ESRI Organisational License (EA...,1,1,(YES),0.9,"The text mentions ESRI Organisational License,...",Does the content of the following text focus o...
294,2021/S 081-209781,Chair of the Board of Directors of the Port Au...,1,1,(YES,80%,) The text focuses on using and providing serv...,Does the content of the following text focus o...
295,2021/S 124-329242,Freedom Square 7|ESRI ArcGIS software purchase...,1,1,(YES,80%,The text mentions the purchase and maintenance...,Does the content of the following text focus o...
296,2022/S 143-410171,Bavarian State Forests AöR|Forest planning 4.0...,1,1,(YES,90%,) The text focuses on using and providing serv...,Does the content of the following text focus o...


# Plots

In [33]:
#import data
import pandas as pd
output_file='./paper.xlsx'
input_data=pd.read_excel(output_file)

import pandas as pd
df_context=pd.read_excel('./paper_gpt_validation_w_context.xlsx')
confirmed=df_context[df_context['chatGPT_labels']==1].id.values

In [34]:
#count terms
results=[]
for col in [col for col in input_data.columns  if 'full' in col ]:
    terms=input_data[(~input_data[col].isna()) &(input_data['id'].isin(confirmed))][col].values

    terms_all=[]
    for el in terms:
        for t in el.split('|'):
            terms_all.append(t)

    terms_all=pd.DataFrame(terms_all)
    terms_all.columns=['terms']
    terms_all['topic']=col.replace('-terms-full','')
    results.append(terms_all)
results=pd.concat(results)
results=results.reset_index()

In [52]:
# plot
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'results' is your initial DataFrame
# Group by 'topic' and 'terms', and count occurrences
df_counts = results.groupby(['topic', 'terms'])['index'].count().reset_index(name='count')

# Function to get top 15 occurrences for each topic
def get_top_15(group):
    return group.nlargest(15, 'count')

# Apply the function to get the top 15 occurrences per topic
top_15_per_topic = df_counts.groupby('topic', group_keys=False).apply(get_top_15)

# Plotting
unique_topics = top_15_per_topic['topic'].unique()

for topic in unique_topics:
    data = top_15_per_topic[top_15_per_topic['topic'] == topic]
    plt.figure(figsize=(12, 6))
    sns.barplot(x='count', y='terms', data=data, palette='viridis')
    plt.title(f'Top 15 Terms for Topic: {topic}')
    plt.xlabel('Count')
    plt.ylabel('Terms')
    plt.subplots_adjust(left=0.3)
    plt.savefig('./plot/%s.png'%(topic),dpi=300)
    plt.close()


In [37]:
topic

'PROPRIETARY GEOSPATIAL SOFTWARE'

In [38]:
unique_topics

array(['GEOSPATIAL', 'GEOSPATIAL STANDARDS', 'GIS OPEN SOURCE SOFTWARE',
       'OPEN DATA', 'OPEN GEOSPATIAL DATASETS', 'OPEN SOURCE SOFTWARE',
       'OPEN STANDARDS', 'PROPRIETARY GEOSPATIAL SOFTWARE'], dtype=object)

In [39]:
top_15_per_topic

,topic,terms,count
11,GEOSPATIAL,gis,73
15,GEOSPATIAL,spatial data,32
10,GEOSPATIAL,geospatial,31
7,GEOSPATIAL,geographical information,19
5,GEOSPATIAL,geographic information,8
...,...,...,...
68,PROPRIETARY GEOSPATIAL SOFTWARE,Autodesk,23
72,PROPRIETARY GEOSPATIAL SOFTWARE,Smallworld,9
71,PROPRIETARY GEOSPATIAL SOFTWARE,MapInfo,7
70,PROPRIETARY GEOSPATIAL SOFTWARE,Geomedia,5


In [50]:
input_data[~input_data['OPEN GEOSPATIAL DATASETS-terms-full'].isna()]['OPEN GEOSPATIAL DATASETS-terms-full']

3887     OpenStreetMap
4022    open road data
4303     OpenStreetMap
5299     OpenStreetMap
Name: OPEN GEOSPATIAL DATASETS-terms-full, dtype: object